<a href="https://colab.research.google.com/github/kavyajeetbora/recipe_recommender/blob/master/notebooks/food_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 997.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=45019ee89a6f616f597fafb3b7be0f2bd4b8cb43ddda8615a1cbc03658b0e346
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
import torch
from tqdm.notebook import tqdm
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read the data
nutrition - calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) ,carbohydrates (PDV)

In [4]:
%%time
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/food_recommender/data_file/RAW_recipes.csv')
df = df.reset_index(drop=True)
df.head()

CPU times: user 4.74 s, sys: 559 ms, total: 5.3 s
Wall time: 10.4 s


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


## Transform the data

In [6]:
## Split the nutrition column to separate columns
df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df.nutrition.str.split(",",expand=True)

## Replace [] brackets after split
df['calories'] =  df['calories'].apply(lambda x: x.replace('[',''))
df['carbohydrates (PDV)'] =  df['carbohydrates (PDV)'].apply(lambda x: x.replace(']',''))

## Convert the datatypes from str to float
df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']].astype('float')

## Drop unnecessary columns
df.drop(['id', 'contributor_id','submitted','tags','nutrition'], axis=1,inplace = True)

In [7]:
df.dtypes

name                    object
minutes                  int64
n_steps                  int64
steps                   object
description             object
ingredients             object
n_ingredients            int64
calories               float64
total fat (PDV)        float64
sugar (PDV)            float64
sodium (PDV)           float64
protein (PDV)          float64
saturated fat (PDV)    float64
carbohydrates (PDV)    float64
dtype: object

In [8]:
df.head()

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,arriba baked winter squash mexican style,55,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,30,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,130,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,45,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,190,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


## Export the data to a parquet file

In [9]:
df.to_parquet('/content/drive/MyDrive/Colab Notebooks/Projects/food_recommender/data_file/RAW_recipes.parquet')

## Cleaning the ingredients column

In [ ]:
%%time
## First convert the string representation of list to list
df['ingredients'] = df['ingredients'].str.strip("[]").str.replace("'","") #.str.split(",")

CPU times: user 317 ms, sys: 22.8 ms, total: 339 ms
Wall time: 340 ms


In [ ]:
df['ingredients'].iloc[0]

'winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt'

Explode the series containing list to a single list:

In [ ]:
ingredients = df['ingredients'].explode().str.lower().str.strip()
print(f"Around {ingredients.shape[0]} ingredients are there in the dataset")

Around 231637 ingredients are there in the dataset


Now find the unique ingredients:

In [ ]:
unique_ingredients = ingredients.unique()
print(f"There around {unique_ingredients.shape[0]} are unique out of {ingredients.shape[0]}")

There around 230475 are unique out of 231637


There might be some same words that mean the same but with different name.

Try with some keywords to see if it exists like search for 'egg' or 'milk'

In [ ]:
keyword = 'bread'
unique_word = ingredients[ingredients.str.contains(keyword.lower())].unique()
if len(unique_word)> 0:
    sample = ", ".join(np.random.choice(unique_word,10))
    print(f"There are around {len(unique_word)} unique ingredients containing the work 'f{keyword}'.\n\n Example:\n\n {sample}....")
else:
    print(f"No ingredients for the keyword: {keyword}")

There are around 18673 unique ingredients containing the work 'fbread'.

 Example:

 milk, condensed cream of celery soup, processed swiss cheese, sauerkraut, butter, rye bread, caraway seed, corned beef, butter, dijon mustard, dried thyme, dried parsley, salt, bread, cheddar cheese, smoked ham, red onion, tomatoes, cooked chicken, chive & onion cream cheese, garlic, frozen chopped spinach, mozzarella cheese, parmesan cheese, breadcrumbs, seasoning salt, black pepper, roasted red peppers, alfredo sauce, cannelloni tubes, granulated yeast, bread flour, salt, sugar, olive oil, water, spring onion, feta cheese, edam cheese, roquefort cheese, fresh dill, fresh breadcrumbs, heavy cream, eggs, olive oil, spinach, frozen bread dough, egg, parsley, oregano, pepper, mozzarella cheese, pepperoni, pizza sauce, carrots, orange, lemon, red wine vinegar, extra virgin olive oil, sea salt, fresh ground black pepper, whole cumin seeds, dried chilies, garlic cloves, fresh thyme leaves, avocados, bread, 

## Sentence encoder model


Selecting the encoder model: https://www.sbert.net/docs/pretrained_models.html

In [17]:
%%time
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

CPU times: user 332 ms, sys: 151 ms, total: 483 ms
Wall time: 499 ms


In [11]:
# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

## Encoding the ingredients

In [18]:
%%time

## encode all the ingredients:
embedding = model.encode(df['ingredients'].values)

CPU times: user 2min 46s, sys: 2.08 s, total: 2min 48s
Wall time: 2min 35s


In [22]:
df_final = df.copy()
df_final['embedding'] = list(zip(embedding))
df_final['embedding'] = df_final['embedding'].apply(lambda x: x[0])
df_final.head()

,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),embedding
0,arriba baked winter squash mexican style,55,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,"[-0.039017964, 0.030269215, 0.010577081, 0.089..."
1,a bit different breakfast pizza,30,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,"[-0.06494449, 0.008968874, 0.059068605, 0.0291..."
2,all in the kitchen chili,130,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,"[-0.10917001, -0.01785316, 0.0396873, 0.044659..."
3,alouette potatoes,45,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,"[-0.07200331, -0.017013816, 0.0010840622, 0.02..."
4,amish tomato ketchup for canning,190,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,"[-0.09281766, 0.047554053, 0.02394141, 0.04069..."


In [23]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   name                 231636 non-null  object 
 1   minutes              231637 non-null  int64  
 2   n_steps              231637 non-null  int64  
 3   steps                231637 non-null  object 
 4   description          226658 non-null  object 
 5   ingredients          231637 non-null  object 
 6   n_ingredients        231637 non-null  int64  
 7   calories             231637 non-null  float64
 8   total fat (PDV)      231637 non-null  float64
 9   sugar (PDV)          231637 non-null  float64
 10  sodium (PDV)         231637 non-null  float64
 11  protein (PDV)        231637 non-null  float64
 12  saturated fat (PDV)  231637 non-null  float64
 13  carbohydrates (PDV)  231637 non-null  float64
 14  embedding            231637 non-null  object 
dtypes: float64(7), in

## Export the embedded recipes to parquet files

In [35]:
partitions = np.linspace(0, df_final.shape[0], 30, dtype=int)
for ix in range(len(partitions)-1):
    i,j = partitions[ix], partitions[ix+1]
    dfx = df_final.iloc[i:j,:]
    dfx.to_parquet(rf'/content/drive/MyDrive/Colab Notebooks/Projects/food_recommender/data_file/Exports/recipes_{ix}.parquet')

## Finally calculate the cosine similarity

In [24]:
def cosine_similarity(vec1,vec2):
    '''
    Returns the cosine similarity between two vectors of n dimension
    '''
    denom = np.sqrt(np.sum(np.square(vec1))) * np.sqrt(np.sum(np.square(vec2)))
    return np.round(np.dot(vec1,vec2) / denom * 100, 2)

In [25]:
index = 3
data = df_final.iloc[index]
recipe, vector = data['name'], data['embedding']

print("Name of the dish:", recipe)

Name of the dish: alouette  potatoes


In [26]:
%%time

df_result = df_final.copy()
df_result['similarity'] = df_result['embedding'].apply(lambda x : cosine_similarity(vector, x))
df_result.sort_values(by="similarity", ascending=False).iloc[1:4]

CPU times: user 9.29 s, sys: 530 ms, total: 9.82 s
Wall time: 10.3 s


,name,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),embedding,similarity
189645,smoked mozzarella and penne salad,30,5,['process the dressing ingredients in food pro...,from the whole foods market cookbook. i buy t...,"['parmesan cheese', 'fresh parsley', 'mayonnai...",11,262.4,19.0,5.0,14.0,20.0,25.0,9.0,"[-0.09707589, -0.01945921, 0.018470693, 0.0582...",92.82
39949,cheesy arrabbiata over linguine,32,6,"['in a large pan , heat 2 tbsp of oil over med...","we love arrabbiata, the hotter the better for ...","['olive oil', 'onions', 'shallot', 'red pepper...",14,557.3,25.0,59.0,15.0,32.0,13.0,29.0,"[-0.09360438, 0.002902513, 0.023475569, 0.0308...",92.73
9704,asparagus and goat cheese salad,2,10,['heat 2 tablespoons oil in 12-inch nonstick s...,not tried yet,"['virgin olive oil', 'red bell pepper', 'aspar...",10,321.7,44.0,9.0,8.0,20.0,43.0,2.0,"[-0.0630732, 0.005144814, 0.0011174467, 0.0333...",92.68
